In [2]:
import pandas as pd
import numpy as np
from time import sleep
import os.path

In [17]:
artigos = pd.read_csv("titulo_doc_comunidade_page_rank_with_id.csv",delimiter=';',quotechar='\"')
# artigos.head()
artigos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 3 columns):
title        428 non-null object
community    428 non-null int64
id           428 non-null object
dtypes: int64(1), object(2)
memory usage: 10.1+ KB


In [5]:
artigos.title.duplicated().sum()

0

In [14]:
artigos['downloaded'] = artigos.id.apply(lambda x: os.path.isfile('../pdf/{}.pdf'.format(x)))

In [15]:
artigos[~artigos.downloaded]

,title,community,id,downloaded
194,Selecting Computations: Theory and Applications,grupo 134,1207.5879v1,False


In [27]:
for ID in artigos[~artigos.downloaded].id.unique():
    link = "https://arxiv.org/pdf/"+ID+".pdf"
    ! cd pdf && curl -s -O {link} 
    sleep(np.random.random_sample()*35)

In [10]:
for ID in artigos.id.unique():
    pdf = "../pdf/{}.pdf".format(ID)
    txt = "../txt/{}.txt".format(ID)
    !pdftotext {pdf} {txt}

^C
^C


Syntax Error: Missing language pack for 'Adobe-GB1' mapping
Syntax Error: Unknown font tag 'R126'
Syntax Error (620285): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (620285): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show
Syntax Error: Unknown font tag 'R126'
Syntax Error (621704): No font in show


^C


Syntax Error: Expected the default config, but wasn't able to find it, or it isn't a Dictionary


##### Remoção das primeiras linhas dos textos para evitar cabeçalho dos artigos

In [5]:
############################## comando para terminal ##############################
# for file in *.txt; do tail -n +20 "$file" -f txt2/"$file"; done

O código a seguir foi adaptado de http://brandonrose.org/clustering#Stopwords,-stemming,-and-tokenizing

In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

In [7]:
############################## Remover comentários dos downloads ao rodar a primeira vez ##############################
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(re.sub('[-\xad][\s\n]','',text)) \
                          for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]{3,}', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
#     return filtered_tokens

[nltk_data] Downloading package stopwords to /home/angelo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/angelo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
tfidf_vectorizer = TfidfVectorizer(input='filename', max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

####################### Pegar somente os arquivos cujos ids estão no arquivo específico de estudo #######################
t = []

for ID in artigos.id.unique():
    t.append("../txt/" + ID + ".txt")
# t = !ls txt2/*.txt
# t = !ls ../txt/*.txt

print(len(t))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(t)

print(tfidf_matrix.shape)

428
CPU times: user 1min 52s, sys: 1.7 s, total: 1min 54s
Wall time: 1min 57s
(428, 4264)


In [25]:
terms = tfidf_vectorizer.get_feature_names()
terms[:10]

['aaai',
 'aaai press',
 'aalborg',
 'aalborg denmark',
 'aalborg univers',
 'abbrevi',
 'abduct',
 'abil',
 'abl',
 'abov']

In [26]:
with open('terms.txt','w') as f:
    f.write('\n'.join(terms))

In [27]:
x = tfidf_matrix.toarray()

In [28]:
df = pd.DataFrame(x,columns=terms,index=[re.sub('../txt?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
df.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,y-axi,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang
1301.2285v1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.029757,0.005490,0.017763,...,0.0,0.000000,0.0,0.0,0.017464,0.000000,0.000000,0.017559,0.012964,0.0
1303.5757v1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.014184,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1401.3453v1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.013715,...,0.0,0.000000,0.0,0.0,0.017978,0.000000,0.000000,0.000000,0.000000,0.0
1303.1518v1,0.0,0.0,0.0,0.0,0.0,0.007567,0.0,0.000000,0.003587,0.005803,...,0.0,0.000000,0.0,0.0,0.003803,0.000000,0.025651,0.030592,0.059290,0.0
1302.6852v1,0.0,0.0,0.0,0.0,0.0,0.029610,0.0,0.000000,0.007017,0.034059,...,0.0,0.000000,0.0,0.0,0.000000,0.009639,0.033459,0.029927,0.000000,0.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 428 entries, 1301.2285v1 to 1301.2262v1
Columns: 4264 entries, aaai to zhang
dtypes: float64(4264)
memory usage: 13.9+ MB


In [30]:
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')
doc_term_matrix.reset_index(inplace=True)
doc_term_matrix.drop('level_0',inplace=True,axis=1)
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.029757,0.005490,0.017763,...,0.0,0.0,0.017464,0.000000,0.000000,0.017559,0.012964,0.0,6,1301.2285v1
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,6,1303.5757v1
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.013715,...,0.0,0.0,0.017978,0.000000,0.000000,0.000000,0.000000,0.0,6,1401.3453v1
3,0.0,0.0,0.0,0.0,0.0,0.007567,0.0,0.000000,0.003587,0.005803,...,0.0,0.0,0.003803,0.000000,0.025651,0.030592,0.059290,0.0,6,1303.1518v1
4,0.0,0.0,0.0,0.0,0.0,0.029610,0.0,0.000000,0.007017,0.034059,...,0.0,0.0,0.000000,0.009639,0.033459,0.029927,0.000000,0.0,6,1302.6852v1


In [31]:
doc_term_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Columns: 4266 entries, aaai to id
dtypes: float64(4264), int64(1), object(1)
memory usage: 13.9+ MB


In [13]:
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,abbrevi,abduct,abil,abl,abov,abov equat,...,year,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.016812,0.0,0.008762,0.0,0.0,grupo 6,1201.2250v1
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.005207,0.0,...,0.000000,0.0,0.003463,0.000000,0.0,0.000000,0.0,0.0,grupo 151,1201.6655v1
2,0.021062,0.000000,0.0,0.0,0.007406,0.0,0.004745,0.003509,0.000000,0.0,...,0.005825,0.0,0.022266,0.000000,0.0,0.000000,0.0,0.0,grupo 134,1202.3698v1
3,0.006916,0.011222,0.0,0.0,0.000000,0.0,0.000000,0.004609,0.000000,0.0,...,0.000000,0.0,0.004874,0.018742,0.0,0.004884,0.0,0.0,grupo 6,1202.3699v1
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.005544,0.028650,0.0,...,0.000000,0.0,0.002931,0.003757,0.0,0.005875,0.0,0.0,grupo 104,1202.3706v1


#### Convertendo os valores dos grupos para números

In [32]:
doc_term_matrix.community.replace(r'[a-zA-Z]* *', value='', inplace=True, regex=True)
doc_term_matrix.community = doc_term_matrix.community.apply(int)
doc_term_matrix.community.unique()

array([  6,  80, 104, 116, 134, 145, 151, 153, 156, 256])

In [33]:
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community,id
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.029757,0.005490,0.017763,...,0.0,0.0,0.017464,0.000000,0.000000,0.017559,0.012964,0.0,6,1301.2285v1
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,6,1303.5757v1
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.013715,...,0.0,0.0,0.017978,0.000000,0.000000,0.000000,0.000000,0.0,6,1401.3453v1
3,0.0,0.0,0.0,0.0,0.0,0.007567,0.0,0.000000,0.003587,0.005803,...,0.0,0.0,0.003803,0.000000,0.025651,0.030592,0.059290,0.0,6,1303.1518v1
4,0.0,0.0,0.0,0.0,0.0,0.029610,0.0,0.000000,0.007017,0.034059,...,0.0,0.0,0.000000,0.009639,0.033459,0.029927,0.000000,0.0,6,1302.6852v1


In [34]:
doc_term_matrix.community.unique()

array([  6,  80, 104, 116, 134, 145, 151, 153, 156, 256])

#### Discretizando os valores

In [35]:
x2 = doc_term_matrix.drop(['id','community'],axis=1).\
                            replace(to_replace=0,value=np.NaN).\
                            apply(lambda a: pd.qcut(a,3,labels=['baixo','medio','alto']))
x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community'] = doc_term_matrix.community
x2.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alto,medio,alto,...,NaN,NaN,NaN,alto,NaN,NaN,alto,medio,NaN,6
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,alto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medio,...,NaN,NaN,NaN,alto,NaN,NaN,NaN,NaN,NaN,6
3,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,baixo,baixo,...,NaN,NaN,NaN,baixo,NaN,medio,alto,alto,NaN,6
4,NaN,NaN,NaN,NaN,NaN,alto,NaN,NaN,medio,alto,...,NaN,NaN,NaN,NaN,medio,alto,alto,NaN,NaN,6


In [36]:
x2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Columns: 4265 entries, aaai to community
dtypes: category(4264), int64(1)
memory usage: 2.5 MB


In [38]:
x2.fillna('zero').to_csv('doc_term_matrix_tfidf_with_header_discrete.csv',header=True,index=False)

In [39]:
x2.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alto,medio,alto,...,NaN,NaN,NaN,alto,NaN,NaN,alto,medio,NaN,6
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,alto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medio,...,NaN,NaN,NaN,alto,NaN,NaN,NaN,NaN,NaN,6
3,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,baixo,baixo,...,NaN,NaN,NaN,baixo,NaN,medio,alto,alto,NaN,6
4,NaN,NaN,NaN,NaN,NaN,alto,NaN,NaN,medio,alto,...,NaN,NaN,NaN,NaN,medio,alto,alto,NaN,NaN,6


In [43]:
x2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Columns: 4265 entries, aaai to community
dtypes: category(4264), int64(1)
memory usage: 2.5 MB


In [44]:
freq_vectorizer = CountVectorizer(input='filename',max_df=0.9, max_features=20000,
                                 min_df=0.05, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
# t = !ls ../txt/*.txt
%time freq_matrix = freq_vectorizer.fit_transform(t)

print(freq_matrix.shape)

CPU times: user 1min 52s, sys: 74.3 ms, total: 1min 52s
Wall time: 1min 52s
(428, 4264)


In [45]:
terms_freq = freq_vectorizer.get_feature_names()

with open('terms_tf.txt','w') as f:
    f.write('\n'.join(terms_freq))

In [46]:
x = freq_matrix.toarray()
df = pd.DataFrame(x,columns=terms_freq,index=[re.sub('../txt?/(\d+.?\d+v\d+)\.txt','\\1',a) for a in t])
doc_term_matrix = pd.merge(df,artigos[['community','id']],left_index=True,right_on='id')

doc_term_matrix.set_index('id',inplace=True)
doc_term_matrix.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
id,,,,,,,,,,,,,,,,,,,,,
1301.2285v1,0,0,0,0,0,0,0,4,1,4,...,0,0,0,3,0,0,3,1,0,6
1303.5757v1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,6
1401.3453v1,0,0,0,0,0,0,0,0,0,10,...,0,0,0,10,0,0,0,0,0,6
1303.1518v1,0,0,0,0,0,1,0,0,1,2,...,0,0,0,1,0,3,8,7,0,6
1302.6852v1,0,0,0,0,0,2,0,0,1,6,...,0,0,0,0,1,2,4,0,0,6


In [47]:
doc_term_matrix.community.replace(r'[a-zA-Z]* *', value='', inplace=True, regex=True)
doc_term_matrix.community = doc_term_matrix.community.apply(int)
doc_term_matrix.community.unique()

array([  6,  80, 104, 116, 134, 145, 151, 153, 156, 256])

In [48]:
doc_term_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 428 entries, 1301.2285v1 to 1301.2262v1
Columns: 4265 entries, aaai to community
dtypes: int64(4265)
memory usage: 13.9+ MB


In [49]:
doc_term_matrix.to_csv('doc_term_matrix_freq_with_stem.csv',header=True)

In [52]:
filtro = doc_term_matrix.drop('community',axis=1)\
                .replace(to_replace=0,value=np.NaN)\
                .apply(lambda a: [-1,0,a.quantile(.75),a.max()])

In [53]:
filtro.head()

aaai               [-1, 0, 2.0, 26.0]
aaai press         [-1, 0, 2.0, 13.0]
aalborg             [-1, 0, 3.0, 7.0]
aalborg denmark     [-1, 0, 2.0, 3.0]
aalborg univers     [-1, 0, 2.0, 3.0]
dtype: object

In [54]:
np.sum(filtro.apply(lambda a: len(np.unique(a))) == 4)

4167

In [55]:
x2 = doc_term_matrix.drop(['community'],axis=1)\
            .loc[:,filtro.apply(lambda a: len(np.unique(a))) == 4]\
            .apply(lambda a: pd.cut(a,
                                    [-1,0,a.replace(to_replace=0,value=np.NaN).quantile(.75),a.max()],
                                    labels=['zero','baixo','alto']))
# x2 = x2.apply(lambda a: a.cat.add_categories(['zero']))
x2['community'] = doc_term_matrix.community

x2.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
id,,,,,,,,,,,,,,,,,,,,,
1301.2285v1,zero,zero,zero,zero,zero,zero,zero,alto,baixo,baixo,...,zero,zero,zero,baixo,zero,zero,baixo,baixo,zero,6
1303.5757v1,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,...,baixo,zero,zero,zero,zero,zero,zero,zero,zero,6
1401.3453v1,zero,zero,zero,zero,zero,zero,zero,zero,zero,alto,...,zero,zero,zero,alto,zero,zero,zero,zero,zero,6
1303.1518v1,zero,zero,zero,zero,zero,baixo,zero,zero,baixo,baixo,...,zero,zero,zero,baixo,zero,baixo,alto,alto,zero,6
1302.6852v1,zero,zero,zero,zero,zero,alto,zero,zero,baixo,alto,...,zero,zero,zero,zero,baixo,baixo,alto,zero,zero,6


In [56]:
x2.aaai.describe()

count      428
unique       3
top       zero
freq       313
Name: aaai, dtype: object

In [57]:
x2.to_csv('doc_term_matrix_freq_with_header_discrete.csv',header=True,index=False)

In [58]:
doc_term_matrix.community.value_counts()

153    82
104    67
6      53
134    48
156    41
80     39
116    32
256    29
151    19
145    18
Name: community, dtype: int64

In [59]:
doc_term_matrix.drop('community',axis=1).apply(lambda x: np.corrcoef(x.values,doc_term_matrix.community.values)[0,1])


aaai                            -0.015642
aaai press                      -0.002459
aalborg                          0.386703
aalborg denmark                  0.353412
aalborg univers                  0.410042
abbrevi                          0.018754
abduct                          -0.122616
abil                            -0.020775
abl                             -0.043357
abov                            -0.029742
abov exampl                     -0.093425
absenc                           0.050880
absent                           0.000344
absolut                         -0.003567
absorb                           0.144763
abstract paper                   0.045578
academ                          -0.007776
academ press                     0.003044
acceler                         -0.027470
accept                           0.058272
access                          -0.076843
accommod                         0.052657
accomplish                      -0.045583
accord                          -0

In [60]:
from scipy.stats import chi2_contingency

chi2_contingency(pd.crosstab(x2.aaai,x2.community).values)

(52.17828029554045,
 3.512773302535552e-05,
 18,
 array([[38.75934579, 28.52102804, 48.99766355, 23.40186916, 35.10280374,
         13.1635514 , 13.89485981, 59.96728972, 29.98364486, 21.20794393],
        [11.02102804,  8.10981308, 13.93224299,  6.65420561,  9.98130841,
          3.74299065,  3.95093458, 17.05140187,  8.52570093,  6.03037383],
        [ 3.21962617,  2.36915888,  4.07009346,  1.94392523,  2.91588785,
          1.09345794,  1.15420561,  4.98130841,  2.49065421,  1.76168224]]))

In [61]:
b = x2.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x2.community))[1])


In [62]:
x2[b.sort_values(ascending=True)[200:500].index.values.tolist() + ['community']]\
    .to_csv('doc_term_matrix_freq_chisq_discrete.csv',index=False)


In [64]:
x2.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
id,,,,,,,,,,,,,,,,,,,,,
1301.2285v1,zero,zero,zero,zero,zero,zero,zero,alto,baixo,baixo,...,zero,zero,zero,baixo,zero,zero,baixo,baixo,zero,6
1303.5757v1,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,...,baixo,zero,zero,zero,zero,zero,zero,zero,zero,6
1401.3453v1,zero,zero,zero,zero,zero,zero,zero,zero,zero,alto,...,zero,zero,zero,alto,zero,zero,zero,zero,zero,6
1303.1518v1,zero,zero,zero,zero,zero,baixo,zero,zero,baixo,baixo,...,zero,zero,zero,baixo,zero,baixo,alto,alto,zero,6
1302.6852v1,zero,zero,zero,zero,zero,alto,zero,zero,baixo,alto,...,zero,zero,zero,zero,baixo,baixo,alto,zero,zero,6


In [63]:
# x3 = pd.read_csv('doc_term_matrix_tfidf_discrete.csv')
x3 = pd.read_csv('doc_term_matrix_tfidf_with_header_discrete.csv')

x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Columns: 4265 entries, aaai to community
dtypes: int64(1), object(4264)
memory usage: 13.9+ MB


In [65]:
x3.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
0,zero,zero,zero,zero,zero,zero,zero,alto,medio,alto,...,zero,zero,zero,alto,zero,zero,alto,medio,zero,6
1,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,...,alto,zero,zero,zero,zero,zero,zero,zero,zero,6
2,zero,zero,zero,zero,zero,zero,zero,zero,zero,medio,...,zero,zero,zero,alto,zero,zero,zero,zero,zero,6
3,zero,zero,zero,zero,zero,baixo,zero,zero,baixo,baixo,...,zero,zero,zero,baixo,zero,medio,alto,alto,zero,6
4,zero,zero,zero,zero,zero,alto,zero,zero,medio,alto,...,zero,zero,zero,zero,medio,alto,alto,zero,zero,6


In [66]:
b3 = x3.drop('community',axis=1).apply(lambda x: chi2_contingency(pd.crosstab(x,x3.community))[1])

In [67]:
b3.head()

aaai               1.087336e-04
aaai press         2.533265e-01
aalborg            2.314009e-22
aalborg denmark    9.148217e-16
aalborg univers    3.398442e-23
dtype: float64

In [77]:
x3[b3.sort_values(ascending=True)[300:1000].index.values.tolist() + ['community']]\
    .to_csv('doc_term_matrix_tfidf_chisq_discrete_300_to_1000.csv',index=False)


In [78]:
x3.head()

,aaai,aaai press,aalborg,aalborg denmark,aalborg univers,abbrevi,abduct,abil,abl,abov,...,year,yedidia,yes,yield,york,zadeh,zero,zero probabl,zhang,community
0,zero,zero,zero,zero,zero,zero,zero,alto,medio,alto,...,zero,zero,zero,alto,zero,zero,alto,medio,zero,6
1,zero,zero,zero,zero,zero,zero,zero,zero,zero,zero,...,alto,zero,zero,zero,zero,zero,zero,zero,zero,6
2,zero,zero,zero,zero,zero,zero,zero,zero,zero,medio,...,zero,zero,zero,alto,zero,zero,zero,zero,zero,6
3,zero,zero,zero,zero,zero,baixo,zero,zero,baixo,baixo,...,zero,zero,zero,baixo,zero,medio,alto,alto,zero,6
4,zero,zero,zero,zero,zero,alto,zero,zero,medio,alto,...,zero,zero,zero,zero,medio,alto,alto,zero,zero,6


In [79]:
x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Columns: 4265 entries, aaai to community
dtypes: int64(1), object(4264)
memory usage: 13.9+ MB
